# Model creation

In [ ]:
aolQueries = []

for i in range(1, 11):
  if i < 10:
    fileName = "drive/MyDrive/EDDS/user-ct-test-collection-0" + str(i) + ".txt"
  else:
    fileName = "drive/MyDrive/EDDS/user-ct-test-collection-" + str(i) + ".txt"

  lines = []
  with open(fileName) as f:
    lines = f.readlines()

  count = 0
  for line in lines:
    if count > 0:
      query = line.split("\t")[1]
      aolQueries.append(query)

    count += 1

In [ ]:
aolTopSuffixes = {}
aolTop2WordSuffixes = {}
aolTop3WordSuffixes = {}

for query in aolQueries:
  words = query.split(" ")
  lastOne = words[-1]

  if lastOne in aolTopSuffixes:
    aolTopSuffixes[lastOne] += 1
  else:
    aolTopSuffixes[lastOne] = 1


  if len(words) >= 2:
    lastTwo = words[-2] + " " + words[-1]

    if lastTwo in aolTop2WordSuffixes:
      aolTop2WordSuffixes[lastTwo] += 1
    else:
      aolTop2WordSuffixes[lastTwo] = 1

  if len(words) >= 3:
    lastThree = words[-3] + " " + words[-2] + " " + words[-1]
    
    if lastThree in aolTop3WordSuffixes:
      aolTop3WordSuffixes[lastThree] += 1
    else:
      aolTop3WordSuffixes[lastThree] = 1

In [ ]:
aolTopSuffixesCombined = {**aolTop2WordSuffixes, **aolTop3WordSuffixes}

aolTopSuffixesSorted = [*dict(sorted(aolTopSuffixesCombined.items(), key=lambda item: item[1]))]
aolTopSuffixesSorted.reverse()

#aolTop2WordSuffixesSorted = [*dict(sorted(aolTop2WordSuffixes.items(), key=lambda item: item[1]))]
#aolTop2WordSuffixesSorted.reverse()

#aolTop3WordSuffixesSorted = [*dict(sorted(aolTop3WordSuffixes.items(), key=lambda item: item[1]))]
#aolTop3WordSuffixesSorted.reverse()

In [ ]:
def findTopNCompletions(topSuffixesSorted, query, n):
  completions = []
  nCompletions = 0

  for suffix in topSuffixesSorted:
    if suffix.startswith(query.split("_")[-1]):
      completions.append(query.rsplit("_", 1)[0] + "_" + suffix.replace(" ", "_"))
      nCompletions += 1

    if nCompletions >= n:
      break

  return completions

# Evaluation


In [ ]:
q0List = []

lines = []
with open("/content/drive/MyDrive/EDDS/QAC-run-AOL") as f:
  lines = f.readlines()

for line in lines:
  lineParts = line.split("\t")
  query = lineParts[0]
  if query not in q0List:
    q0List.append(query)

In [ ]:
with open("/content/drive/MyDrive/EDDS/PopSuffic-AOL-Results", "w") as f:
  for q0 in q0List:
    completions = findTopNCompletions(aolTopSuffixesSorted, q0, 20)

    rank = 1
    for c in completions:
      f.write(q0 + "\t" + "Q0" + "\t" + c + "\t" + str(rank) + "\t" + "0" + "\t" + "AOLPopSuffix")
      f.write('\n')

      rank += 1

## Evaluation from ecir2018-neuqs repo

In [ ]:
from collections import defaultdict

import numpy as np

import sys





def read_trec_gt(path):

	""" read Trec ground truth file to get a dictionary 

	{q1:[s1, s2, ...], q2:[s, ...], ....}

	where q is query, [s1, s2, ...] are relevant suggestions

	"""

	relevance_gt_dict = defaultdict(list)

	with open(path, "r", encoding="latin1") as f:

		for line in f:

			query, _, suggestion, _, r, _ = line.strip().split("\t")

			if r == "1" or r == "2":

				relevance_gt_dict[query].append(suggestion)

	return relevance_gt_dict





def read_gt(path):

	""" read ground truth file to get a dictionary 

	{q1:[s1, s2, ...], q2:[s, ...], ....}

	where q is query, [s1, s2, ...] are relevant suggestions

	"""

	relevance_gt_dict = defaultdict(list)

	with open(path, "r", encoding="latin1") as f:

		for line in f:

			query, _, suggestion, r = line.strip().split("\t")

			if r == "1":

				relevance_gt_dict[query].append(suggestion)

	return relevance_gt_dict





def read_run(path):

	""" read run file to get a dictionary 

	{q1:[s1, s2, ...], q2:[s, ...], ....}

	where q is query, [s1, s2, ...] are ranked suggestions

	"""

	run_dict = defaultdict(list)

	with open(path, "r", encoding="latin1") as f:

		for line in f:

			query, _, suggestion, _, _, _ = line.strip().split("\t")

			run_dict[query].append(suggestion)

	return run_dict





def cal_precision_at_k(rel_suggestions, gen_suggestions, k):

	"""given relevant suggestions and candidate suggestions for a query,

	calculate precision at k.

	"""

	# precisions a list of precisions, the ith element (precisions[i]) 

	# is the precision at (i+1)-th position

	# rel_count counting the number of relevant candidate suggestions

	precision, rel_count = 0, 0

	k_gen_suggestions = gen_suggestions[:k]



	# iter all candidate suggestions

	for i in range(len(k_gen_suggestions)):

		# get current suggestion

		gen_s = k_gen_suggestions[i]

		# if current suggestion appear in relevant suggestions

		if gen_s in rel_suggestions:

			# relevant suggestion count +1

			rel_count += 1.0



	precision = rel_count/k



	return precision





def eval_run(run_dict, relevance_gt_dict, run_name, k):

	""" evaluate each run

	

	- run_dict, a dictionray {q1:[s1, s2, ...], q2:[s, ...], ....}

	where q is query, [s1, s2, ...] are ranked suggestions

	

	- relevance_gt_dict, {q1:[s1, s2, ...], q2:[s, ...], ....}

	where q is query, [s1, s2, ...] are relevant suggestions in groundtruth

	

	- run_name, name of this run

	

	- k, precision at position k

	"""

	p_list, lens = [], []



	for query, results in run_dict.items():

		# evaluate the results of this query

		precision = cal_precision_at_k(relevance_gt_dict[query], results, k)

		l = len(results)

		

		p_list.append(precision)	

		lens.append(l)

	print("P@"+str(k)+":\t"+str(np.sum(p_list)/100)+"\t"+run_name)

	# print("P@"+str(k)+":\t"+str(np.sum(p_list)/100)+"\t"+str(len(p_list))+"\t"+run_name)




In [ ]:
relevance_gt_dict = read_gt(path="/content/drive/MyDrive/EDDS/QAC_ground_truth.tsv")
run_dict = read_run(path="/content/drive/MyDrive/EDDS/PopSuffic-AOL-Results")

In [ ]:
k = 20
eval_run(run_dict, relevance_gt_dict, "PopSuffic-AOL-Results", k)

P@20:	0.21150000000000002	PopSuffic-AOL-Results


In [ ]:
k = 10
eval_run(run_dict, relevance_gt_dict, "PopSuffic-AOL-Results", k)

P@10:	0.255	PopSuffic-AOL-Results
